In [ ]:
import pyspark
from pyspark.sql import SparkSession

## Creating a Spark Session

In [ ]:
spark = SparkSession.builder.getOrCreate()
spark

#### Reading a csv file with spark

In [ ]:
df = spark.read.format("csv").option("header","true").load(r"original.csv")

In [ ]:
df.show(5)

There are null present in some columns of the dataframe before moving to the next analysis need to handle them.

In [ ]:
from pyspark.sql.functions import *

In [ ]:
## Created a new column named clean_city in which replacing null values which are present in city column with "UnKnown"
df1 = df.withColumn("Clean_City",when(df.City.isNull(), 'UnKnown').otherwise(df.City))

In [ ]:
df1.show(5)

In [ ]:
## Filtering the data frame with the records where there are no null values present in the Jobtitle
df1 = df1.filter(df1.JobTitle.isNotNull())

In [ ]:
df1.show(5)

In [ ]:
## Replacing the null value of salary column with mean of salary column
df1 = df1.withColumn('Clean_Salary',df1.Salary.substr(2,100).cast('float'))

In [ ]:
df1.show(5)

In [ ]:
mean = df1.groupBy().avg('Clean_Salary').take(1)[0][0]

In [ ]:
mean

In [ ]:
from pyspark.sql.functions import lit

In [ ]:
df1 = df1.withColumn('new_salary', when(df1.Clean_Salary.isNull(), lit(mean)).otherwise(df1.Clean_Salary))

In [ ]:
df1.show(5)

In [ ]:
## populate the median value of latitude and longitude
import numpy as np

In [ ]:
lat = df1.select("Latitude")

In [ ]:
lat.show(5)

In [ ]:
lat = lat.filter(lat.Latitude.isNotNull())

In [ ]:
lat.show()

In [ ]:
lat = lat.withColumn('latitude2',lat.Latitude.cast('float')).select('latitude2')

In [ ]:
lat.show(5)

In [ ]:
medain = np.median(lat.collect())

In [ ]:
medain

In [ ]:
df1 = df1.withColumn('lat_new', when(df1.Latitude.isNull(), lit(medain)).otherwise(df1.Latitude))

In [ ]:
df1.show(5)

In [ ]:
#Q1 - Overall the mean or women get paid more on average?
#Q2 - By Job Title do men or women get paid more on average?
#Q3 - By Which City has the highest average Salary?

#### Answering Q1


In [ ]:
import pyspark.sql.functions as sqlfunc

In [ ]:
genders = df1.groupBy('gender').agg(sqlfunc.avg('new_salary').alias('AvgSalary'))

In [ ]:
genders.show()

### Answering Q2

In [ ]:
df2 = df1.withColumn('female_salary', when(df1.gender == 'Female', df1.new_salary).otherwise(lit(0)))

In [ ]:
df2 = df2.withColumn('male_salary', when(df2.gender == 'Male', df2.new_salary).otherwise(0))

In [ ]:
df2.show(5)

In [ ]:
df2 = df2.groupBy("JobTitle").agg(sqlfunc.avg('female_salary').alias('final_female_salary'), sqlfunc.avg('male_salary').alias('final_male_salary'))

In [ ]:
df2.show()

In [ ]:
df2 = df2.withColumn('Delta', df2.final_female_salary - df2.final_male_salary)

In [ ]:
df2.show()

### Answering Q3

In [ ]:
cityavg = df1.groupBy('City').agg(sqlfunc.avg('new_salary').alias('avgsalary'))

In [ ]:
cityavg = cityavg.sort(col('avgsalary').desc())

In [ ]:
cityavg.show()

### Bringing Data into Dataframes

In [ ]:
## Reading the csv in another way in spark
df_n = spark.read.csv(r'original.csv', header = True)

In [ ]:
df_n.show(5)

In [ ]:
# data types of the attributes present in the dataframe
df_n.dtypes

As the spark interpretted every column as string eventhough there are some interger or float attribute present need to change them from string to thier respective data type

In [ ]:
# changing the data types of the attribues.
from pyspark.sql.types import *
# As one can define their schema in sql here also we can  define the schema prior itself of the data which we will be 
# loading in the spark session. So the schema goes likes this

schema = StructType([StructField('id', IntegerType()),
                     StructField('first_name', StringType()),
                     StructField('last_name', StringType()),
                     StructField('gender',StringType()),
                     StructField('City', StringType()),
                     StructField('JobTitle', StringType()),
                     StructField('Salary', StringType()),
                     StructField('Latitude', StringType()),
                     StructField('Longitude', FloatType())])

df4 = spark.read.csv(r'original.csv', header = True, schema = schema)

In [ ]:
df4.show(5)

In [ ]:
df4.dtypes

In [ ]:
df4_n = df4.withColumn('Latitude', df4.Longitude.cast('float'))

In [ ]:
df4_n.dtypes

### Inspecting a DataFrame

In [ ]:
schema = StructType([StructField('id', IntegerType()),
                     StructField('first_name', StringType()),
                     StructField('last_name', StringType()),
                     StructField('gender',StringType()),
                     StructField('City', StringType()),
                     StructField('JobTitle', StringType()),
                     StructField('Salary', StringType()),
                     StructField('Latitude', FloatType()),
                     StructField('Longitude', FloatType())])

df_nn = spark.read.csv(r'original.csv', header = True, schema = schema)

In [ ]:
df_nn.dtypes

In [ ]:
df_nn.head(5)

In [ ]:
df_nn.first()

In [ ]:
df_nn.describe().show()

In [ ]:
df_nn.columns

In [ ]:
df_nn.distinct().count()

In [ ]:
df_nn.count()

### Handling Nulls and Duplicate Values

In [ ]:
## dropping all the records from the data where ever nulls are present in any of the column
df_dropped = df_nn.na.drop()

In [ ]:
df_dropped.show(5)

In [ ]:
## Dropping the records from the dataset where nulls are present in the jobtitle column
df_not_null_jobs = df_nn.filter(df_nn.JobTitle.isNotNull())

In [ ]:
df_not_null_jobs.show()

In [ ]:
## Creating a new column if there is a null present in a column replace null with unknown or keep what is was there.
df_handled = df_nn.withColumn("Clean_City", when(df_nn.City.isNull(),"Unknown").otherwise(df_nn.City))

In [ ]:
df_handled.show()

In [ ]:
## Dropping the duplicates from the dataset
df_no_duplicates = df_nn.dropDuplicates()

In [ ]:
df_no_duplicates.show()

### Selecting and filtering Data

In [ ]:
## Select particular columns from the dataframe
df_nn.show(2)

Let's Select first_name and Last_name from the above dataframe

In [ ]:
df_select = df_nn.select("first_name","last_name")
df_select.show()

In [ ]:
##Renaming a column
df_rename_firstname = df_nn.withColumnRenamed('first_name','fn')
df_rename_firstname.show(3)

In [ ]:
## filtering the dataframe for a particular attribute from a particular column
df_filter = df_nn.filter(df_nn.first_name == 'Alvera')
df_filter.show()

In [ ]:
## filtering the dataframe with a wildcard expression from a particular column
df_filter_we = df_nn.filter(df_nn.first_name.like('%lver%'))
df_filter_we.show()

In [ ]:
## filtering a dataframe based on  a particular column where the values endwith a particular pattern
df_filter_ew = df_nn.filter(df_nn.first_name.endswith('din'))
df_filter_ew.show()

In [ ]:
## filtering a dataframe based on  a particular column where the values endwith a particular pattern
df_filter_sw = df_nn.filter(df_nn.first_name.startswith('Alv'))
df_filter_sw.show()

In [ ]:
### filtering a numerical column between certain range of values
df_filter_btw = df_nn.filter(df_nn.id.between(1,5))
df_filter_btw.show()

In [ ]:
### filtering the dataframe based on more than one attribute from a particular column.
df_filter_fnm = df_nn.filter((df_nn.first_name.isin('Aldin','Valma')))
df_filter_fnm.show()

In [ ]:
## breaking the values of a particular column to certain characters and creating a new column with it.(Substring)
df_subs = df_nn.select(df_nn.first_name, df_nn.first_name.substr(1,5).alias('New_Name'))
df_subs.show(5)

### Applying multiple Filter

In [ ]:
## scenario - 1
df_filt1 = df_nn.filter((df_nn.first_name.isin('Aldin','Valma')) | (df_nn.City.like("%ondon")))
df_filt1.show()

In [ ]:
# Scenario - 2
df_filt2 = df_nn.filter((df_nn.id > 10) & (df_nn.id < 100))
df_filt2.show(5)

### Running SQL on DataFrames

In [ ]:
df_nn.registerTempTable('original')

In [ ]:
query1 = spark.sql('select * from original')
query1.show(5)

In [ ]:
query2 = spark.sql('select concat(first_name," ",last_name) as full_name from original')
query2.show(5)

In [ ]:
df_nn.show(1)

### Adding Calculated Columns

In [ ]:
df_cc = df_nn.withColumn('Clean_Salary', df_nn.Salary.substr(2,100).cast('float'))
df_cc.show(3)

In [ ]:
df_mn_slr = df_cc.withColumn('Monthly_Salary', df_cc.Clean_Salary/12)
df_mn_slr.show(5)

In [ ]:
## Adding a new column to identify whether the customer is a female or not
df_fc = df_nn.withColumn("Are_they_Female", when(df_nn.gender == 'Female', 'Yes').otherwise('No'))
df_fc.show(5)

### Groupby and Aggregation

In [ ]:
## Groupby gender and get the sum of total salary
df_g_s = df_cc.groupBy('gender').agg(sqlfunc.sum('Clean_Salary'))

In [ ]:
df_g_s.show()

In [ ]:
## Groupby gender and get the sum, avg, min and max of clean_salary
df_gp_cs = df_cc.groupBy('gender').agg(sqlfunc.sum('Clean_Salary').alias("Total"),
                                       sqlfunc.avg('Clean_Salary').alias("Average"),
                                       sqlfunc.max('Clean_Salary').alias("Maximum"),
                                       sqlfunc.min('Clean_Salary').alias("Minimum"))

df_gp_cs.show()

In [ ]:
## Groupby gender, city and get the sum, avg, min and max of clean_salary

df_g_c_cs = df_cc.groupBy('gender','city').agg(sqlfunc.sum("Clean_Salary").alias("Total"),
                                               sqlfunc.avg("Clean_Salary").alias("Average"),
                                               sqlfunc.max("Clean_Salary").alias("Maximum"),
                                               sqlfunc.min("Clean_Salary").alias("Minimum"))

In [ ]:
df_g_c_cs.show()

In [ ]:
## Wrting output to a file

df_nn.write.csv('df_nn.csv')